In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import numpy as np
import plotly.graph_objects as go
from scipy.spatial import ConvexHull

import pandas as pd
import matplotlib.pyplot as plt

# Generate random 3D points with labels
np.random.seed(42)
x = np.random.randn(100)
y = np.random.randn(100)
z = np.random.randn(100)

# Assign random labels
labels = np.random.choice([0, 1, 2, 3], size=100)

representatives = ["Fry Cook", "Personal Trainer", "Bus Drive", "Teacher"]

user_x, user_y = [0.5, 0, 0], 2

# df = pd.DataFrame(np.append(np.random.randn(100,3), labels), columns=["x","y","z","label"])
df = pd.DataFrame(np.random.randn(100,3), columns=["x", "y", "z"])

X = df.values

df["label"] = labels


In [12]:

# Create the Dash app
app = dash.Dash(__name__)

# Create a 3D-hull scatter plot given covariates X (n x 3) and labels y (n x 1)
def create_figure(X, y, repr, p_x, p_y):

    # Array containing all the clusters and their convex hulls
    layers = []

    unique_labels = np.unique(y)

    k = len(unique_labels)

    color_palette = plt.get_cmap("plasma")(np.linspace(0, 1, k))  # Using colormap from Matplotlib
    colors = [f'rgba({int(c[0] * 255)}, {int(c[1] * 255)}, {int(c[2] * 255)}, {c[3]})' for c in color_palette]

    user_point = go.Scatter3d(
                x=[p_x[0]], y=[p_x[1]], z=[p_x[2]], 
                mode='markers', 
                marker=dict(
                    size=10,
                    color=colors[p_y],
                    symbol='cross'),
                name=f'Label {repr[p_y]}',
                legendgroup=f'Label {repr[p_y]}',
                showlegend=False,
            )
    layers.append(user_point)

    # Create scatter plots and convex hull volumes for each label in y
    for label in unique_labels:
        # select rows with corresponding label
        X_k = X[y==label]

        # Compute the convex hull for this group if it has enough points
        if X_k.shape[0] >= 4:
            hull = ConvexHull(X_k)

            x_c, y_c, z_c  = X_k[:,0], X_k[:,1], X_k[:,2] 

            # Scatter plot for points
            scatter = go.Scatter3d(
                x=x_c, y=y_c, z=z_c, 
                mode='markers', 
                marker=dict(size=5, color=colors[label], opacity=1),
                name=f'Label {label}',
                legendgroup=f'Label {label}',
                showlegend=False,
            )
            layers.append(scatter)

            # Mesh3d for convex hull
            mesh = go.Mesh3d(
                x=x_c,
                y=y_c,
                z=z_c,
                i=hull.simplices[:, 0],
                j=hull.simplices[:, 1],
                k=hull.simplices[:, 2],
                opacity=0.3,
                color=colors[label],
                hoverinfo='text',
                hovertext=f'{repr[label]}',
                name=f'Hull {label}',
                legendgroup=f'Hull {label}'
            )
            layers.append(mesh)

    # Create figure
    fig = go.Figure(data=layers)
    fig.update_layout(plot_bgcolor='lightblue',
                      paper_bgcolor='darkgray',
                      scene=dict(aspectmode='data',
                                xaxis=dict(visible=False),
                                yaxis=dict(visible=False),
                                zaxis=dict(visible=False)),
                      title="Job Space",
                    )

    return fig

# Layout of the app
app.layout = html.Div([
    dcc.Graph(id='3d-scatter', figure=create_figure(X, labels, representatives, user_x, user_y)),
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8080)


Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


AttributeError: 'tuple' object has no attribute 'tb_frame'